# Intro to Convolutional Neural Networks

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
dataTransforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(15),
        transforms.RandomAffine(0,shear=8, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
    ])
}

trainset = datasets.CIFAR10(root='./data',
                                      train=True,
                                      download=True,                                     
                                      transform=dataTransforms['train'])
trainloader = DataLoader(trainset, batch_size = 16, shuffle=True)

testset = datasets.CIFAR10(root='./data',
                                     train=False,
                                     download=True,
                                     transform=dataTransforms['test'])
testloader = DataLoader(testset,batch_size=16,shuffle=True)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

def showImg(img):
    plt.imshow(np.transpose(img/2 + 0.5,(1,2,0)))
    plt.show()
dataiter = iter(trainloader)
images, labels = dataiter.next()
showImg(torchvision.utils.make_grid(images))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

#using vgg16 as inspiration to test out using a cnn 
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1_1 = nn.Conv2d(3,64,3,padding=1)
        self.conv1_2 = nn.Conv2d(64,64,3,padding=1)
        
        self.conv2_1 = nn.Conv2d(64,256,3,padding=1)
        self.conv2_2 = nn.Conv2d(256,256,3,padding=1)
        
        self.pool = nn.MaxPool2d(2,2)
        
        self.dropout = nn.Dropout(0.15)
        
        self.fc1 = nn.Linear(256*8*8,256)
        self.fc2 = nn.Linear(256,64)
        self.fc3 = nn.Linear(64,10)
   
        
    def forward(self,x):
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool(x)
        
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool(x)
                              
        x = torch.flatten(x,1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
                      
        return x
        
model = Model()
model.to(device)

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.003,momentum=0.9)

In [ ]:
epochs = 15
for epoch in range(epochs):
    #train
    model.train()
    trainLoss = 0
    for images, labels in trainloader:
        images,labels = images.to(device),labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        trainLoss += loss.item()*images.size(0)
    #test
    model.eval()
    correct = 0
    testLoss = 0
    total = 0
    with torch.no_grad():
        for images,labels in testloader:
            images,labels = images.to(device),labels.to(device)
            output = model(images)
            loss = criterion(output,labels)
            testLoss += loss.item()*images.size(0)
            _, prediction = torch.max(output.data,1)
            total += labels.size(0)
            correct += (prediction==labels).sum().item()
    #print results
    accuracy = (correct*100 / total)
    trainLoss/=(len(trainloader) * 16) #average loss (16 is batch size)
    testLoss/= (len(testloader)*16)
    print("Epoch: {}, Train Loss: {:.3f}, Test Loss: {:.3f}, Accuracy: {:.3f}".format(epoch,trainLoss,testLoss,accuracy))
    